##1. 분류

- 분류(Classification): 학습데이터로 범주를 학습해 모델을 생성하고,새로운 데이터 값이 주어졌을 때 미지의 범주값을 예측

- 결정 트리(Decision Tree, 의사결정트리, 의사결정나무라고도 함)는 분류(Classification)와 회귀(Regression) 모두 가능한 지도 학습 모델.

 - if/else 로 스무고개?

 - 매, 펭귄, 돌고래, 곰 분류
<figure>
    <img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FwlH1u%2FbtqwWZI9Xen%2FkFJDjGSFJAPxhyatC3Xhs0%2Fimg.png' width=700 height=300>
</figure>

 - 특정 기준(질문)에 따라 데이터를 구분하는 모델

<figure>
    <img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fdy5OwG%2FbtqDwdHofoT%2FNtDy9lqXkhWTRTwEz6txd0%2Fimg.png' width=700 height=300>
</figure>

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

data = pd.read_csv("creditset2.csv")
X = data.drop("default10yr", axis=1)
y = data.default10yr
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = DecisionTreeClassifier()
model.fit( X_train, y_train )
pred = model.predict( X_test )

print( classification_report(y_test, pred))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


              precision    recall  f1-score   support

           0       0.99      1.00      0.99       440
           1       0.96      0.92      0.94        60

    accuracy                           0.99       500
   macro avg       0.98      0.96      0.97       500
weighted avg       0.99      0.99      0.99       500



##2. 군집

- Unsupervised Learning: Y 변수가 없는 데이터에서 패턴을 발견
- Clustering: 거리가 가까운 관측치들 간 유사한 특징이 있을 것이라는 가정을 기반으로 행들을 군집화
  - 인사이트 탐색 용도
  - 1) 계층적 클러스터링 (hierarchical clustering)과 2) 비계층적 클러스터링(K-means clustering)으로 구분됨
    - 1) 거리행렬을 기반으로 가까운 대상끼리 순차적으로 그룹을 할당
    - 2) 랜덤한 시작점에서 해당 점과 가까운 대상 간 그룹 할당

- 이상치가 존재할 경우 클러스터링 진행 시 영향을 많이 받을 수 있으므로 사전에 제거 과정이 필요

In [54]:
from sklearn.preprocessing import StandardScaler
X = data.drop("default10yr", axis=1)
X_scaled = StandardScaler().fit_transform(X)
X_scaled_df = pd.DataFrame( X_scaled, columns=X.columns)

from sklearn.cluster import AgglomerativeClustering, KMeans
kc = KMeans()
kc.fit(X_scaled_df)
print(kc.labels_)
print(kc.cluster_centers_)
print(kc.inertia_)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[3 7 3 ... 2 5 3]
[[ 0.78596402  0.81525723 -0.60138362]
 [ 1.06012061 -0.91274273  1.49143263]
 [-0.18850229 -0.97503002  0.45956184]
 [ 1.07069461  0.91581419  1.53689457]
 [-1.12075423 -0.7740458  -0.73631812]
 [-1.03885357  0.94923531 -0.65649837]
 [ 0.8569606  -0.94459814 -0.67504298]
 [-0.07757117  0.68710489  0.57670996]]
1355.7065783258893


## 3. 패턴 발견
- 연관 규칙 분석: Apriori
 - Rule 기반, 장바구니 분석
 - 지지도/신뢰도/향상도

 - 지지도(Support)

 $~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Support = P(A {\cap} B) $

                          품목 A와 품목 B를 포함하는 거래수 / 전체 거래 수

 - 신뢰도(Confidence)

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Confidence = P(B | A) = {\dfrac {P(A {\cap} B)}{P(A)}} $

                          항목 A의 거래중에서 항목 B가 포함된 거래의 비율
                          지지도(Support) / P(A)

 - 향상도(Lift:improvement)

  - A ${\rightarrow}$ B 의 연관 규칙에서 임의로(Random) B가 구매되는 경우에 비해 A와의 관계가 고려되어 구매되는 경우의 비율

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Lift = {\dfrac{P(B|A)}{P(B)}} = {\dfrac{P(A{\cap}B)}{P(A)P(B)}} $

                          신뢰도(Confidence) / P(B)

 - 장점
  - 원리가 간단하여 사용자가 쉽게 이해할 수 있고 의미를 파악할 수 있음
  - 유의한 연관성을 갖는 구매패턴을 찾아줌

- 단점
 - 데이터가 클 경우 (item이 많은 경우)에 속도가 느리고 연산량이 많음
 - 실제 사용시에 많은 연관상품들이 나타나는 단점이 있음


- **FP-Growth**
 - Apriori의 속도 개선
 - FP Tree라는 구조를 사용
 - 빈번하게 발생하는 아이템 셋(frequent itemsets)을 효과적으로 발견
 - 아이템 간의 연관성을 찾는 것의 어려움: Confidence 등의 계산

- 장점
 - Apriori 알고리즘보다 빠른 탐색
 - 후보 Itemsets 을 생성할 필요없이 진행 가능

- 단점
 - Apriori 알고리즘에 비해서 설계하기 어려움
 - 지지도의 계산이 FP-Tree가 만들어지고 나서야 가능함

In [55]:
import mlxtend
print(mlxtend.__version__)

0.23.1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
import mlxtend
import numpy as np
import pandas as pd

data = [['우유', '기저귀', '쥬스'],['양상추', '기저귀', '맥주'],['우유', '양상추', '기저귀',  '맥주'],['양상추','맥주']]
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['우유', '기저귀', '쥬스'],
 ['양상추', '기저귀', '맥주'],
 ['우유', '양상추', '기저귀', '맥주'],
 ['양상추', '맥주']]

In [57]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(data).transform(data) #array
df = pd.DataFrame(te_ary, columns=te.columns_)
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


기저귀     맥주    양상추     우유     쥬스
0   True  False  False   True   True
1   True   True   True  False  False
2   True   True   True   True  False
3  False   True   True  False  False

In [58]:
from mlxtend.frequent_patterns import apriori
apriori(df, min_support=0.5, use_colnames=True) #최소기준을 만족한 패턴들을 모두 생성

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


support        itemsets
0     0.75           (기저귀)
1     0.75            (맥주)
2     0.75           (양상추)
3     0.50            (우유)
4     0.50       (기저귀, 맥주)
5     0.50      (기저귀, 양상추)
6     0.50       (기저귀, 우유)
7     0.75       (맥주, 양상추)
8     0.50  (기저귀, 맥주, 양상추)

In [59]:
from mlxtend.frequent_patterns import fpgrowth
fpgrowth(df, min_support=0.5, use_colnames=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


support        itemsets
0     0.75           (기저귀)
1     0.50            (우유)
2     0.75           (양상추)
3     0.75            (맥주)
4     0.50       (기저귀, 맥주)
5     0.50      (기저귀, 양상추)
6     0.50  (기저귀, 맥주, 양상추)
7     0.50       (기저귀, 우유)
8     0.75       (맥주, 양상추)

- data3.csv에 적용

In [60]:
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd

df = pd.read_csv('data_3.csv')
df.tail()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


user                                        artist  qty
19994  1312                                  the national    1
19995  1312                                    bloc party    1
19996  1312                   the jimi hendrix experience    1
19997  1312                                         cream    1
19998  1312  ...and you will know us by the trail of dead    1

In [61]:
df['artist'] = df['artist'].str.strip()#앞뒤 공백 제거
df.dropna(axis=0, subset=['user'], inplace=True)
df['user'] = df['user'].astype('str')		#문자열로 변환
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


user                                        artist  qty
0         1                         red hot chili peppers    1
1         1                       the black dahlia murder    1
2         1                                     goldfrapp    1
3         1                              dropkick murphys    1
4         1                                      le tigre    1
...     ...                                           ...  ...
19994  1312                                  the national    1
19995  1312                                    bloc party    1
19996  1312                   the jimi hendrix experience    1
19997  1312                                         cream    1
19998  1312  ...and you will know us by the trail of dead    1

[19999 rows x 3 columns]

In [62]:
df.pivot_table(index='user', columns='artist', values='qty').fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


artist  ...and you will know us by the trail of dead  2pac  3 doors down  \
user                                                                       
1                                                0.0   0.0           0.0   
1000                                             0.0   0.0           0.0   
1002                                             0.0   0.0           0.0   
1005                                             0.0   0.0           0.0   
1007                                             0.0   0.0           0.0   
...                                              ...   ...           ...   
992                                              0.0   0.0           0.0   
993                                              0.0   1.0           0.0   
995                                              0.0   0.0           0.0   
996                                              0.0   0.0           0.0   
999                                              0.0   1.0           0.0   

artist  30 seconds to mars  311  36 crazyfists   44  50 cent  65daysofstatic  \
user                                                                           
1                      0.0  0.0            0.0  0.0      0.0             0.0   
1000                   0.0  0.0            0.0  0.0      0.0             0.0   
1002                   0.0  0.0            0.0  0.0      0.0             0.0   
1005                   0.0  0.0            0.0  1.0      0.0             0.0   
1007                   0.0  0.0            0.0  0.0      0.0             0.0   
...                    ...  ...            ...  ...      ...             ...   
992                    0.0  0.0            0.0  0.0      0.0             0.0   
993                    0.0  0.0            0.0  0.0      0.0             0.0   
995                    0.0  0.0            0.0  0.0      0.0             0.0   
996                    0.0  0.0            0.0  0.0      0.0             0.0   
999                    0.0  0.0            0.0  0.0      0.0             0.0   

artist  [unknown]  ...  wilco  within temptation  wolfgang amadeus mozart  \
user               ...                                                      
1             0.0  ...    0.0                0.0                      0.0   
1000          0.0  ...    0.0                0.0                      0.0   
1002          0.0  ...    0.0                0.0                      0.0   
1005          0.0  ...    0.0                0.0                      0.0   
1007          0.0  ...    0.0                0.0                      0.0   
...           ...  ...    ...                ...                      ...   
992           0.0  ...    0.0                0.0                      0.0   
993           0.0  ...    0.0                0.0                      0.0   
995           0.0  ...    0.0                0.0                      0.0   
996           0.0  ...    0.0                0.0                      0.0   
999           0.0  ...    0.0                0.0                      0.0   

artist  wu-tang clan  yann tiersen  yeah yeah yeahs  yellowcard  yo la tengo  \
user                                                                           
1                0.0           0.0              0.0         0.0          0.0   
1000             0.0           0.0              0.0         0.0          0.0   
1002             0.0           0.0              0.0         0.0          0.0   
1005             0.0           0.0              0.0         0.0          0.0   
1007             0.0           0.0              0.0         0.0          0.0   
...              ...           ...              ...         ...          ...   
992              0.0           0.0              0.0         0.0          0.0   
993              0.0           0.0              0.0         0.0          0.0   
995              0.0           0.0              0.0         0.0          0.0   
996              0.0           0.0              0.0         0.0          0.0   
999     

In [63]:
basket=df.pivot_table(index='user', columns='artist', values='qty').fillna(0)

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['...and you will know us by the trail of dead', '2pac', '3 doors down',
       '30 seconds to mars', '311', '36 crazyfists', '44', '50 cent',
       '65daysofstatic', '[unknown]',
       ...
       'wilco', 'within temptation', 'wolfgang amadeus mozart', 'wu-tang clan',
       'yann tiersen', 'yeah yeah yeahs', 'yellowcard', 'yo la tengo',
       'zero 7', 'Édith piaf'],
      dtype='object', name='artist', length=1004)

In [64]:
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()
rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.1) ]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


antecedents    consequents  antecedent support  consequent support  \
0     (coldplay)    (radiohead)            0.149357            0.201780   
1    (radiohead)     (coldplay)            0.201780            0.149357   
2  (the beatles)    (radiohead)            0.190900            0.201780   
3    (radiohead)  (the beatles)            0.201780            0.190900   

    support  confidence      lift  leverage  conviction  zhangs_metric  
0  0.065282    0.437086  2.166147  0.035145    1.418014       0.632875  
1  0.065282    0.323529  2.166147  0.035145    1.257472       0.674440  
2  0.071217    0.373057  1.848827  0.032697    1.273193       0.567441  
3  0.071217    0.352941  1.848827  0.032697    1.250427       0.575176

In [65]:
frequent_itemsets2 = fpgrowth(basket_sets, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)
rules2.head()
rules2[ (rules2['lift'] >= 1) & (rules2['confidence'] >= 0.1) ]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


antecedents    consequents  antecedent support  consequent support  \
0     (coldplay)    (radiohead)            0.149357            0.201780   
1    (radiohead)     (coldplay)            0.201780            0.149357   
2  (the beatles)    (radiohead)            0.190900            0.201780   
3    (radiohead)  (the beatles)            0.201780            0.190900   

    support  confidence      lift  leverage  conviction  zhangs_metric  
0  0.065282    0.437086  2.166147  0.035145    1.418014       0.632875  
1  0.065282    0.323529  2.166147  0.035145    1.257472       0.674440  
2  0.071217    0.373057  1.848827  0.032697    1.273193       0.567441  
3  0.071217    0.352941  1.848827  0.032697    1.250427       0.575176

 -chipotle의 데이터로 연관규칙 발견

In [66]:
import pandas as pd
df = pd.read_csv("chipotle.tsv", delimiter='\t', header=0)
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


order_id  quantity                              item_name  \
0            1         1           Chips and Fresh Tomato Salsa   
1            1         1                                   Izze   
2            1         1                       Nantucket Nectar   
3            1         1  Chips and Tomatillo-Green Chili Salsa   
4            2         2                           Chicken Bowl   
...        ...       ...                                    ...   
4617      1833         1                          Steak Burrito   
4618      1833         1                          Steak Burrito   
4619      1834         1                     Chicken Salad Bowl   
4620      1834         1                     Chicken Salad Bowl   
4621      1834         1                     Chicken Salad Bowl   

                                     choice_description item_price  
0                                                   NaN     $2.39   
1                                          [Clementine]     $3.39   
2                                               [Apple]     $3.39   
3                                                   NaN     $2.39   
4     [Tomatillo-Red Chili Salsa (Hot), [Black Beans...    $16.98   
...                                                 ...        ...  
4617  [Fresh Tomato Salsa, [Rice, Black Beans, Sour ...    $11.75   
4618  [Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...    $11.75   
4619  [Fresh Tomato Salsa, [Fajita Vegetables, Pinto...    $11.25   
4620  [Fresh Tomato Salsa, [Fajita Vegetables, Lettu...     $8.75   
4621  [Fresh Tomato Salsa, [Fajita Vegetables, Pinto...     $8.75   

[4622 rows x 5 columns]

In [67]:
df_tmp = df.pivot_table(index='order_id', columns='item_name', values='quantity').fillna(0)
df_tmp = df_tmp.applymap(encode_units)
df_tmp

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


item_name  6 Pack Soft Drink  Barbacoa Bowl  Barbacoa Burrito  \
order_id                                                        
1                          0              0                 0   
2                          0              0                 0   
3                          0              0                 0   
4                          0              0                 0   
5                          0              0                 0   
...                      ...            ...               ...   
1830                       0              0                 0   
1831                       0              0                 0   
1832                       0              0                 0   
1833                       0              0                 0   
1834                       0              0                 0   

item_name  Barbacoa Crispy Tacos  Barbacoa Salad Bowl  Barbacoa Soft Tacos  \
order_id                                                                     
1                              0                    0                    0   
2                              0                    0                    0   
3                              0                    0                    0   
4                              0                    0                    0   
5                              0                    0                    0   
...                          ...                  ...                  ...   
1830                           0                    0                    0   
1831                           0                    0                    0   
1832                           0                    0                    0   
1833                           0                    0                    0   
1834                           0                    0                    0   

item_name  Bottled Water  Bowl  Burrito  Canned Soda  ...  Steak Crispy Tacos  \
order_id                                              ...                       
1                      0     0        0            0  ...                   0   
2                      0     0        0            0  ...                   0   
3                      0     0        0            0  ...                   0   
4                      0     0        0            0  ...                   0   
5                      0     0        0            0  ...                   0   
...                  ...   ...      ...          ...  ...                 ...   
1830                   0     0        0            0  ...                   0   
1831                   1     0        0            0  ...                   0   
1832                   0     0        0            0  ...                   0   
1833                   0     0        0            0  ...                   0   
1834                   0     0        0            0  ...                   0   

item_name  Steak Salad  Steak Salad Bowl  Steak Soft Tacos  Veggie Bowl  \
order_id                                                                  
1                    0                 0                 0            0   
2                    0                 0                 0            0   
3                    0                 0                 0            0   
4                    0                 0                 1            0   
5                    0                 0                 0            0   
...                ...               ...               ...          ...   
1830                 0                 0                 0            0   
1831                 0                 0                 0            0   
1832                 0                 0                 0            0   
1833                 0                 0                 0            0   
1834                 0                 0                 0            0   

item_name  Veggie Burrito  Veggie Crispy Tacos  Veggie Salad  \
order_id                                     

In [68]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit_transform(df_tmp) # df_tmp를 장바구니 형태로 변경 (fit과 transform을 동시에 진행)
df = pd.DataFrame(te_ary, columns=te.columns_) # label 정보는 te.columns_에 저장되어 있으므로, 해당 값을 칼럼명으로 하는 데이터 프레임으로 변경.
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-      6      B      C      D      F      G      I      M  ...  \
0  True  False   True  False  False   True  False  False  False  False  ...   
1  True  False  False   True  False  False  False  False  False  False  ...   
2  True  False  False   True  False  False  False  False  False  False  ...   
3  True  False  False   True   True  False  False  False  False  False  ...   
4  True  False  False   True  False  False  False  False  False  False  ...   

       n     o      p     r      s      t      u      w      y      z  
0   True  True  False  True  False   True  False  False  False  False  
1  False  True  False  True  False  False  False   True  False  False  
2  False  True  False  True  False   True   True  False  False  False  
3  False  True   True  True   True  False  False  False   True  False  
4  False  True  False  True  False  False  False   True  False  False  

[5 rows x 39 columns]

In [69]:
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True) # 여기서는 0.05를 기준으로 선정
frequent_itemsets.sort_values(by='support', ascending=False)

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1).sort_values(by = ['confidence', 'lift', 'support'], ascending=False) # 최소 confidence를 0.1로 지정
association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values(by = [ 'lift', 'confidence', 'support'], ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


antecedents consequents  antecedent support  consequent support  \
500       (a, l)         (d)            0.012541            0.012541   
505          (d)      (a, l)            0.012541            0.012541   
1880   (o, a, C)   (s,  , i)            0.012541            0.012541   
1893   (s,  , i)   (o, a, C)            0.012541            0.012541   
918    ( , a, l)         (d)            0.011996            0.012541   
...          ...         ...                 ...                 ...   
100          (o)         (l)            0.022901            0.014177   
327       ( , o)         (l)            0.021810            0.014177   
101          (l)         (o)            0.014177            0.022901   
492          (S)      (i, o)            0.016358            0.017993   
489       (i, o)         (S)            0.017993            0.016358   

       support  confidence       lift  leverage  conviction  zhangs_metric  
500   0.010905    0.869565  69.338374  0.010748    7.570520       0.998095  
505   0.010905    0.869565  69.338374  0.010748    7.570520       0.998095  
1880  0.010905    0.869565  69.338374  0.010748    7.570520       0.998095  
1893  0.010905    0.869565  69.338374  0.010748    7.570520       0.998095  
918   0.010360    0.863636  68.865613  0.010209    7.241367       0.997444  
...        ...         ...        ...       ...         ...            ...  
100   0.011450    0.500000  35.269231  0.011126    1.971647       0.994420  
327   0.010905    0.500000  35.269231  0.010596    1.971647       0.993311  
101   0.011450    0.807692  35.269231  0.011126    5.080916       0.985619  
492   0.010360    0.633333  35.197980  0.010066    2.678200       0.987747  
489   0.010360    0.575758  35.197980  0.010066    2.318585       0.989392  

[1918 rows x 10 columns]